### Import Required Libraries and Set Up Environment Variables

In [12]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [13]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [14]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&q={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"


In [15]:
# Create an empty list to store the reviews
reviews_list = []


# loop through pages 0-19
for page in range(0, 20):
    print(f"Retrieving page {page}")


    # create query with a page number
    query_url = f"{query_url}&page={page}"

    # API results show 10 articles at a time
    

    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url).json()

    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)


    
    # Try and save the reviews to the reviews_list
    try:
       
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in response["response"]["docs"]:
            reviews_list.append(review)
       

        # Print the page that was just retrieved
        print(f"Retrieved page {page}")
    except:


        # Print the page number that had no results then break from the loop
        print(f"No results for page {page}")
    break



Retrieving page 0
No results for page 0


In [16]:
# Preview the first 5 results in JSON format
print(json.dumps(reviews_list[:5], indent=4))

# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))



[]
[]


In [17]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()



""


In [19]:
# Check if 'headline.main' column exists in 'reviews_df'
if 'headline.main' in reviews_df.columns:
    reviews_df["title"] = reviews_df["headline.main"].str.extract(r"\u2018(.*?)\u2019.*Review")
else:
    # Handle the case when 'headline.main' column does not exist
    print("Error: 'headline.main' column does not exist in 'reviews_df'")



Error: 'headline.main' column does not exist in 'reviews_df'


In [20]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)



KeyError: 'keywords'

In [22]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df["title"].to_list()
titles_list


KeyError: 'title'

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []



# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0


# Loop through the titles
for title in titles_list:


    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print("Sleeping for 12 seconds")
        time.sleep(12)


    # Add 1 to the request counter
    request_counter += 1

    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string).json()


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:

        # Get movie id
        movie_id = response["results"][0]["id"]


        # Make a request for a the full movie details


        # Execute "GET" request with url
        movie_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}").json()


        
        # Extract the genre names into a list
        genres = []
        for genre in movie_response["genres"]:
            genres.append(genre["name"])


        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in movie_response["spoken_languages"]:
            spoken_languages.append(language["english_name"])


        # Extract the production_countries' name into a list
        production_countries = []
        for country in movie_response["production_countries"]:
            production_countries.append(country["name"])


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_response["title"],
            "genre": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries,
            "release_date": movie_response["release_date"],
            "runtime": movie_response["runtime"],
            "vote_average": movie_response["vote_average"],
            "overview": movie_response["overview"]
        })

        
        # Print out the title that was found
        print(f"processing {request_counter}: {title}")
    except:
        # Print out the title that was not found
        print(f"{title} not found") 




Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found.
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
F

In [ ]:
# Preview the first 5 results in JSON format
print(json.dumps(tmdb_movies_list[:5], indent=4))
# Use json.dumps with argument indent=4 to format data


[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 0.708,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "production_countries": [
            "Argentina"
        ]
    },
    

In [ ]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,0,2021-10-07,4.000,2,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,0,2023-03-24,7.643,21,[Canada]
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy, Adventure]",en,[English],https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.369,96,0,2023-04-21,6.389,122,[United States of America]
3,Other People's Children,Les Enfants des autres,0,[Drama],fr,[French],https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",16.246,104,0,2022-09-21,6.900,137,[France]
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,37820,2023-04-07,6.681,36,"[Germany, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,0,2017-01-11,5.617,184,[United States of America]
151,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,0,2017-01-14,6.051,108,"[Belgium, France]"
152,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,0,2006-05-15,6.708,12,[United Kingdom]
153,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[English, Turkish]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",12.097,111,413844,2017-03-10,6.150,236,"[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_movies_df, on="title", how="inner")
merged_df.dropna()

    


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[],,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,[Drama],en,[English],https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,In Search of Fellini,In Search of Fellini,0,"[Drama, Adventure]",en,[English],,"Lucy, a small-town girl from Ohio, discovers t...",4.103,93,...,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
121,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
122,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
123,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genre", "spoken_languages", "production_countries"]



# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]



# Loop through the list of columns to fix
for column in columns_to_fix:


    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)



    # Loop through characters to remove
    for character in characters_to_remove:


        # Replace the character in the column
        merged_df[column] = merged_df[column].str.replace(character, "")


# Display the fixed DataFrame
merged_df.head()



,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [ ]:
# Drop "byline.person" column
merged_df.drop(columns=["byline.person"], inplace=True)
merged_df[['title', 'headline.main', 'web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count', 'genre', 'spoken_languages', 'production_countries', 'release_date', 'runtime', 'vote_average', 'overview']].head()




In [ ]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df[['title', 'headline.main', 'web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count', 'genre', 'spoken_languages', 'production_countries', 'release_date', 'runtime', 'vote_average', 'overview']].head()



,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [21]:
# Export data to CSV without the index
merged_df.to_csv("output/nyt_movie_reviews.csv", index=False)



NameError: name 'merged_df' is not defined